# Для начала познакомимся с доступными данными

data1 = о купленных билетах (идентификатор мероприятия; идентификатор участника; дата покупки; стоимость билета).

data2 = возрасте и регионе участника программы (возраст — от 14 до 22, регион — один из 85 регионов РФ).

data3 = о мероприятиях в регионе посещения (тип организации; идентификатор мероприятия)

data4 = о факте посещения мероприятий (идентификатор мероприятия; идентификатор участника; дата посещения)

# SQL

До того, как мы обьеденим данные в единую таблицу с помощью SQL, мы можем создать некоторые признаки на этом этапе(правда мы можем их же создать и на этапе Feature selection с помощью Pandas).

Например создать признак ['количество людей на мероприятии'] с помощью групировки по ['идентификатор мероприятия'] и суммированием ['идентификатор участника'] из data4. Остальное сделаем в Pandas

Лучшим способом обьеденения таблиц в единую data, будет к data4 добавлять data1 и data3 по ['идентификатор мероприятия'] с помощью LEFT JOIN. Соответственно каждая строка будет отображать мероприятия посещенные каждым учасником. 

Мы бы могли добавить к этой data таблицу data2, если бы в ней содержалась информация ['идентификатор участника']. Если считать, что участники ходят на мероприятия в их регионе, это бы дало нам возможность создать новые признаки, такие как:
- ['средний возраст учасников мероприятия'] для дальнейшей тагретированной рекомендации участникам в одной возрастной группе; 
- ['средний восраст учасников мероприятий каждой организации']
- ['самый дорогой билет этой возрастной групы на этом мероприятии']
- ['количество учасников мероприятия с такой же возрастной категорией, как у участника']
- ['Количество мероприятий в этом регионе, которые провела эта организация']
- ['общее количество мероприятий в регионе'] 
- ['общее количество типов организации мероприятий в регионе'] 
- ['среднее количество мероприятий в регионе'] среднее количество мероприятий в месяц(неделю);
- ['среднее количество проданых билетов в регионе'] можно и "минимальное" и "максимальное" количество проданных билетов в этом регионе
- ['средняя цена билета в регионе'] можно не только "средняя цена", но и минимальная/максимальная цена билета

# Данные в сгенерированном DataFrame

Так как у нас нет общего ключа, чтобы обьеденить data2 в data, общее количество признаков DataFrame таково:
- ['идентификатор мероприятия']
- ['тип организации']
- ['идентификатор участника']
- ['дата покупки']
- ['стоимость билета']
- ['дата посещения']
- ['количество людей на мероприятии'] новосозданный признак в SQL

## Целевая переменная

В качестве **целевой переменной** для участника, можно создать бинарный признак **['interested']**, который будет '1' во всех случаях нашего датасета. В качестве '0' будут выступать мероприятия в которых  участник не принимал участие - мы сгенерируем их сами и таким образов вдвое увеличим выборку. С помощью моделей классификации мы сможем не только узнать прогноз интереса ['interested'] учасника к мероприятию, но и так же его вероятность.

*Так же, можно взять за целевую переменную "доход от проданных билетов" для организации проводящей мероприятие, прогнозирование которой сведется к модели линейной регресии.*

Возможно это можно сделать с помощью класстеризации (участник принадлежит к кластеру), я еще в предвкушении изучения этой темы))
- по кластеру принадлежности к определенной возрастной категории, 
- по географичесому положению в конкретном регионе, 
- по ценовой категории билета,  
- предпочтения посещать крупные/маленькие мероприятия,
- предпочтения мероприятиям определенного организатора.

# EDA

Проведем первичную разведку данных на пропуски и выбросы. Признак с пропусками 20-25% лучше удалить чтобы исключить отклонение результатов модели(можно сделать на следующем шаге). Выбросы оценим в кадом типе признака отдельно. 

Проверим каждый признак на уникальность данных и его распределение. Определим какие из признаков numerous, categorical, binary, а так же признаки time&data

## Числовые признаки

Построим графики распределения для каждого вида признака. Numerous признаки которые распределены не нормально можно прологарифмировать. Посмотрим boxplot графики (делаем выводы).

Возможно некоторые числовые признаки имеет слысл сгенерировать в категории.

Пропуски можно заполнить медианой или модой. Так же можно создать новый бинарный признак на наличие пропуска "1" либо его отсутсвие "0" для каждого признака в котором много пропусков.

Посмотрим график корреляции числовых признаков. Взять "на карандаш" высококоррелируемые между собой

## Категориальные признаки

Построим графики распределения для каждого признака. Так же графики зависимости одной переменной от другой.

Пропуски можно заполнить или модой лио присвоить новую категорию вместо пропуска. Так же можно создать новый бинарный признак на наличие пропуска "1" либо его отсутсвие "0" для каждого признака в котором много пропусков.

Посмотрим график корреляции категориальных признаков. Взять "на карандаш" высококоррелируемые между собой

## Бинарные признаки

Построим графики распределения для каждого признака. Так же графики зависимости одной переменной от другой.

При пропусках в бинарных признаках, учше всего удалить строки.

Если один бинарный признак высоко коррелирует с другим, скорее всего, с помощью них можно сгенерировать новый категориальный признак, который будет отражать суть двух бинарных(для сокращения размерности матрицы).

# Добавление строк

На данный момент **обучение без учителя** еще в моей будущей програме обучения SkillFactory, чего я уже жду не дождусь)) На данный момент предложу мой вариант решения проблемы с которой столкнулись.

Было бы идеально, если бы у нас были данные о прошлых отказов этих клиентов на проведенные мероприятия. Но это не так, мы переходим к предложеному плану Б.

Сгенерируем строки мероприятий, которые данный учасник не посещал и не покупал на них билет (упоминалось выше во время определения целевой переменной). Количество гренерированных строк с "0" должно быть таким же как и исходных строк в датафрейме с "1" для того, чтобы целевая переменная ['interested'] была сбалансирована.

# Посмотрим на метрики наивной модели

Создадим модель LogisticRegression, SVC(probability=True), RandomForestClassifier и DecisionTreeClassifier  и проверим метрику ROC_AUC если нам нужно узнать вероятность принадлежности учасника к заинтересованости к данному мероприятию. Так же добавим метрики confusion_matrix и f1_score. Позже добавим еще.

# Feature engineering

Признаки time&data ['дата покупки'] и ['дата посещения'] приведем к to_datetime

Сгенерируем признаки на их основе:
- ['Отсчет дней от даты первой покупки']
- ['Отсчет дней от даты первого мероприятия']
- ['Отсчет дней от даты первого мероприятия']
- ['Номер дня недели покупки билета']
- ['Номер дня недели мероприятия']
- ['Номер недели покупки билета']
- ['Номер недели мероприятия']
- ['Является ли день мероприятия выходным или праздником'] - бинарный 1 и 0
- ['Разница дней между покупкой и датой посещения']
- ['Общее количество проданых билетов на это мероприятие']
- ['Заполняемость мероприятия в %']
- ['Доход от продаж билетов на это мероприятие']
- ['Средняя цена билета на это мероприятие']
- ['Количество мероприятий, которые посетил участник с начала отсчета датасета']
- ['Количество мероприятий этой организации, которые участник посетил с начала отсчета датасета']
- ['Количество мероприятий, которые посетил участник в этом же месяце прошлого года']
- ['Количество мероприятий, которые провела эта организация']
- ['Максимальная цена билета на это мероприятие']
- ['Минимальная цена билета на это мероприятие']
- ['Средняя цена билета на это мероприятие']
- ['Нормализированя цена билета на это мероприятие'] - вычисляемо с помощью 3х признаков выше
- ['Количество мероприятий в этом месяце']

Заново переобозначим все признаки в data по типу (['идентификатор мероприятия'] и ['идентификатор участника'], ['дата покупки'] и ['дата посещения'] не имеют важности для модели и мы от них избавимся)

Теперь трансформируем наши бинарные и категориальные признаки с помощью LabelEncoder и OneHotEncoder

Стандартизируем числовые признаки с помощью StandardScaler

Определим границы выросов и трансформируем их на число крайней границы на усах

##  Оценка значимости переменных и корреляция

Оценим переменные по их влиянию на целевую переменную.

Почистим датасет от высоко коррелируемых признаков между соббой, а так же признаки, которые имеют очень маленькую корреляцию с целевой переменной

# Обучение и тест моделей

Мы можем оценивать результаты прогнозирования модели на таких метриках: ROC_AUC, confusion_matrix, f1_score (Так как данные сбалансированы, можно добавить accuracy_score, precision_score, recall_score, precision_recall_curve)

Обучим и протестируем модели LogisticRegression, SVC(probability=True), DecisionTreeClassifier и RandomForestClassifier на этих данных.

*Для предсказания вероятности можно попробовать такие модели как XGBoost, LightGBM, CatBoost, Keras. Но, есть мнение, что они не откалиброваны и нам можно использовать IsotonicRegression для калибровки вероятности.*

Запишим результат для сравнения с результатом применения гиперпараметров.

# Обучение и тест моделей с гиперпараметрами

Благодаря перебору по сетке GridSearchCV либо RandomizedSearchCV, найдем подходящие гиперпараметры к моделям протестированных выше и дополним ими наши модели. Посмотрим результат.

Выберем модель с найлучшим результатам в среднем по метрикам и сгенерирум результаты прогнозирования на новых данных.

# Выводы и ответы на вопросы

**Опишите, как бы вы использовали эти данные для предсказания, какое событие заинтересует участника?**

В качестве целевой переменной для участника, можно создать бинарный признак ['interested'], который будет '1' во всех случаях нашего датасета. В качестве '0' будут выступать мероприятия в которых  участник не принимал участие - мы сгенерируем их сами и таким образов вдвое увеличим выборку. С помощью моделей классификации мы сможем не только узнать прогноз интереса ['interested'] учасника к мероприятию, но и так же его вероятность.

Так же, можно взять за целевую переменную "доход от проданных билетов" для организации проводящей мероприятие, прогнозирование которой сведется к модели линейной регресии.

Возможно это можно сделать с помощью класстеризации (участник принадлежит к кластеру по примеру ниже), я еще в предвкушении изучения этой темы))

    по кластеру принадлежности к определенной возрастной категории,
    по географичесому положению в конкретном регионе,
    по ценовой категории билета,
    предпочтения посещать крупные/маленькие мероприятия,
    предпочтения мероприятиям определенного организатора.



**Какие дополнительные данные могут повлиять на точность предсказаний?**

Было печально увидеть, что в data2 нет признака ['идентификатор участника'] по которому мы могли бы присоеденить информацию о возрасте и регионе участника (так же невозможность создать новые признаки на их основе, что более подробно было описано в **разделе SQL** в этом ноутбуке).


Список предложеных дополнительных признаком ниже (выделены те, которые, по моему мнению, принесут самый большой вклад моделе):
- ['Отсчет дней от даты первой покупки']
- ['Отсчет дней от даты первого мероприятия']
- ['Отсчет дней от даты первого мероприятия']
- ['Номер дня недели покупки билета']
- ['Номер дня недели мероприятия']
- ['Номер недели покупки билета']
- ['Номер недели мероприятия']
- **['Является ли день мероприятия выходным или праздником']** - бинарный 1 и 0
- ['Разница дней между покупкой и датой посещения']
- **['Общее количество проданых билетов на это мероприятие']**
- ['Заполняемость мероприятия в %']
- ['Доход от продаж билетов на это мероприятие']
- **['Средняя цена билета на это мероприятие']**
- **['Количество мероприятий, которые посетил участник с начала отсчета датасета']**
- **['Количество мероприятий этой организации, которые участник посетил с начала отсчета датасета']**
- **['Количество мероприятий, которые посетил участник в этом же месяце прошлого года']**
- ['Количество мероприятий, которые провела эта организация']
- ['Максимальная цена билета на это мероприятие']
- ['Минимальная цена билета на это мероприятие']
- **['Средняя цена билета на это мероприятие']**
- **['Нормализированя цена билета на это мероприятие']** - вычисляемо с помощью 3х признаков выше
- ['Количество мероприятий в этом месяце']


**Какую бы метрику использовали для оценки качества работы рекомендательной модели?**

Вместо бинарного предсказания - заинтересуется ли участник нашим предложенным концертом; мы спрогнозируем **вероятность** его заинтересованости посетить предложеное мероприятие. Позже с помощью регуляции threshold менеджмент бизнеса сможет сам регулировать порог рекомендаций.

Во время представления используемых моделей выше (в разделе **Обучение и тест моделей**), я предпочитаю использовать метрики ROC_AUC, confusion_matrix, f1_score с выбраными моделями LogisticRegression, SVC(probability=True), DecisionTreeClassifier и RandomForestClassifier.
(Так как данные сбалансированы, можно добавить accuracy_score, precision_score, recall_score, precision_recall_curve)

Для предсказания вероятности можно попробовать такие модели как XGBoost, LightGBM, CatBoost, Keras. Но, есть мнение, что они не откалиброваны и нам можно использовать IsotonicRegression для калибровки вероятности.